In [1]:
from minicons import cwe

import torch
import csv

In [2]:
model = cwe.CWE('gpt2')

In [3]:
model.tokenize("The cat ate the rat")

['The', 'Ġcat', 'Ġate', 'Ġthe', 'Ġrat']

In [2]:
base_sentence = "He caught the pass and scored another touchdown. There was nothing he enjoyed more than a good game of"
words = ["football", "baseball", "monopoly"]

In [5]:
models = ["bert-base-uncased", "bert-large-uncased", "roberta-base", "gpt2", "openai-gpt", "xlnet-base-cased", "roberta-large"]

with open("/home/kmisra/contextual_n400.csv", 'w') as f:
    writer = csv.writer(f)
    for model in models:
        transformer = cwe.CWE(model, all_layers = True)
        for l in range(0, transformer.layers + 1):
            for word in words:
                sentence = base_sentence + " " + word
                w, c = transformer.context_cosine(sentence, word, layer = l)
                contextual_similarity = round(torch.tensor(list(dict(zip(w,c)).values())).mean().item(), 4)
                writer.writerow([model, l, word, contextual_similarity])

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [5]:
dict(zip(w,c))

{'[CLS]': 0.9372750520706177,
 'he': 0.9298704862594604,
 'caught': 0.9448925852775574,
 'the': 0.9309799671173096,
 'pass': 0.9259966611862183,
 'and': 0.9368696808815002,
 'scored': 0.906683623790741,
 'another': 0.9423209428787231,
 'touchdown': 0.9199809432029724,
 '.': 0.9390969276428223,
 'there': 0.9411791563034058,
 'was': 0.9454566240310669,
 'nothing': 0.9330489635467529,
 'enjoyed': 0.9498648047447205,
 'more': 0.9263346791267395,
 'than': 0.9219998717308044,
 'a': 0.9281640648841858,
 'good': 0.9335828423500061,
 'game': 0.9325376749038696,
 'of': 0.9373526573181152,
 '[SEP]': 0.932860255241394}